# 使用帮助

tkitAutoTokenizerPosition

In [47]:
from transformers import AutoTokenizer,AutoModel,BertForMaskedLM,AutoModelForMaskedLM
# tokenizer = AutoTokenizer.from_pretrained("hfl/rbtl3")
# from torch.utils.data import DataLoader, random_split,TensorDataset
from transformers import BertTokenizer, AlbertModel

# https://huggingface.co/uer
# tokenizer = BertTokenizer.from_pretrained("clue/albert_chinese_tiny")
# tokenizer = BertTokenizer.from_pretrained("bert-base-chinese")
tokenizer = BertTokenizer.from_pretrained("uer/chinese_roberta_L-8_H-256")

In [48]:
tokenizer.pad_token_id

0

In [49]:
from tkitAutoTokenizerPosition import AutoTokenizerPosition

dir(AutoTokenizerPosition)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'autoLen',
 'autoTypeWord',
 'find_all',
 'fixPosition',
 'getWordList']

In [50]:
tkitToken=AutoTokenizerPosition(tokenizer)
tkitToken

In [51]:
text="性胰腺炎@有 研    究显示，进行早期 ERC"

tkitToken.getWordList(text)

['性',
 '胰',
 '腺',
 '炎',
 '@',
 '有',
 '研',
 '究',
 '显',
 '示',
 '，',
 '进',
 '行',
 '早',
 '期',
 'er',
 '##c']

In [52]:
tkitToken.getWordList(text)

['性',
 '胰',
 '腺',
 '炎',
 '@',
 '有',
 '研',
 '究',
 '显',
 '示',
 '，',
 '进',
 '行',
 '早',
 '期',
 'er',
 '##c']

# 开始使用测试数据

In [53]:
Data="data/data.json"

In [54]:
!pip install tkitJson

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [55]:
import tkitJson
from tqdm import tqdm
datas=[]
for file in [Data]:
    Tjson=tkitJson.Json(file)
    APo=AutoTokenizerPosition(tokenizer)
    for i,it in tqdm(enumerate(Tjson.load())):
        # print(it)
    #     info.find('a')
        data=[]
        for one in it['spo_list']:
            word=one['subject']
            wType=one['subject_type']
            for s_start,s_end in APo.fixPosition(it['text'],word):
#             print(s_start,s_end)
                WordList=APo.getWordList(it['text'])
                data.append({"start":s_start,"end":s_end,"type":wType})  

            word=one['object']["@value"]
            wType=one['object_type']["@value"]
            for s_start,s_end in APo.fixPosition(it['text'],word):
#             print(s_start,s_end)
                WordList=APo.getWordList(it['text'])
                data.append({"start":s_start,"end":s_end,"type":wType})  
    
            word=one['predicate']
            wType="关系"
            for s_start,s_end in APo.fixPosition(it['text'],word):
                WordList=APo.getWordList(it['text'])
                data.append({"start":s_start,"end":s_end,"type":wType})  
#             break
#         print(data)
        datas.append({"text":it['text'],"wordList":WordList,"tag":data,"data":it})
#         if i>10:
# #             print
#             break


print(datas[:1])


8it [00:00, 250.06it/s]

[{'text': '急性胰腺炎@有研究显示，进行早期 ERCP （24 小时内）可以降低梗阻性胆总管结石患者的并发症发生率和死亡率； 但是，对于无胆总管梗阻的胆汁性急性胰腺炎患者，不需要进行早期 ERCP。', 'wordList': ['急', '性', '胰', '腺', '炎', '@', '有', '研', '究', '显', '示', '，', '进', '行', '早', '期', 'er', '##cp', '（', '24', '小', '时', '内', '）', '可', '以', '降', '低', '梗', '阻', '性', '胆', '总', '管', '结', '石', '患', '者', '的', '并', '发', '症', '发', '生', '率', '和', '死', '亡', '率', '；', '但', '是', '，', '对', '于', '无', '胆', '总', '管', '梗', '阻', '的', '胆', '汁', '性', '急', '性', '胰', '腺', '炎', '患', '者', '，', '不', '需', '要', '进', '行', '早', '期', 'er', '##cp', '。'], 'tag': [{'start': 0, 'end': 5, 'type': '疾病'}, {'start': 65, 'end': 70, 'type': '疾病'}, {'start': 16, 'end': 18, 'type': '检查'}, {'start': 80, 'end': 82, 'type': '检查'}], 'data': {'text': '急性胰腺炎@有研究显示，进行早期 ERCP （24 小时内）可以降低梗阻性胆总管结石患者的并发症发生率和死亡率； 但是，对于无胆总管梗阻的胆汁性急性胰腺炎患者，不需要进行早期 ERCP。', 'spo_list': [{'Combined': False, 'predicate': '影像学检查', 'subject': '急性胰腺炎', 'subject_type': '疾病', 'object': {'@value': 'ERCP'}, 'object_type': {'@value': '检查'}}]}}]


查看测试结果

In [46]:


for it in datas:
    print(it)
    tags=["O"]*len(it["wordList"])
    for w,t in zip(it["wordList"],tags):
        # print(w,t)
        
        for tse in it['tag']:
            # print(tse)
            
            for rid in range(tse['start'],tse['end']):
                if len(it["wordList"])>tse['start'] and len(it["wordList"])>tse['end']:
                    if rid==tse['start']:
                        tags[rid]="B-"+tse['type']
                    elif rid ==tse['end']-1:
                        try:
                            tags[rid]="E-"+tse['type']
                        except:
                            print(len(tags),tse['start'],tse['end'])
                    else:
                        tags[rid]="I-"+tse['type']
        
        
        for w,tag in zip(it["text"],tags):
            print(w,tag)

{'text': '急性胰腺炎@有研究显示，进行早期 ERCP （24 小时内）可以降低梗阻性胆总管结石患者的并发症发生率和死亡率； 但是，对于无胆总管梗阻的胆汁性急性胰腺炎患者，不需要进行早期 ERCP。', 'wordList': ['急', '性', '胰', '腺', '炎', '@', '有', '研', '究', '显', '示', '，', '进', '行', '早', '期', 'er', '##cp', '（', '24', '小', '时', '内', '）', '可', '以', '降', '低', '梗', '阻', '性', '胆', '总', '管', '结', '石', '患', '者', '的', '并', '发', '症', '发', '生', '率', '和', '死', '亡', '率', '；', '但', '是', '，', '对', '于', '无', '胆', '总', '管', '梗', '阻', '的', '胆', '汁', '性', '急', '性', '胰', '腺', '炎', '患', '者', '，', '不', '需', '要', '进', '行', '早', '期', 'er', '##cp', '。'], 'tag': [{'start': 0, 'end': 5, 'type': '疾病'}, {'start': 65, 'end': 70, 'type': '疾病'}, {'start': 16, 'end': 18, 'type': '检查'}, {'start': 80, 'end': 82, 'type': '检查'}], 'data': {'text': '急性胰腺炎@有研究显示，进行早期 ERCP （24 小时内）可以降低梗阻性胆总管结石患者的并发症发生率和死亡率； 但是，对于无胆总管梗阻的胆汁性急性胰腺炎患者，不需要进行早期 ERCP。', 'spo_list': [{'Combined': False, 'predicate': '影像学检查', 'subject': '急性胰腺炎', 'subject_type': '疾病', 'object': {'@value': 'ERCP'}, 'object_type': {'@value': '检查'}}]}}
急 B-疾病
性 I-疾